In [ ]:
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertPreTrainedModel, BertModel
from transformers import get_linear_schedule_with_warmup

from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForMaskedLM,XLMRobertaForSequenceClassification

In [ ]:
model_name = 'sagorsarker/bangla-bert-base'#model name
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)#tokenizer
model = BertForSequenceClassification.from_pretrained(model_name,num_labels=5)#model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
import pandas as pd
df=pd.read_csv("")#datapath
stop_words_df = pd.read_excel('/kaggle/input/bangla-stopwords/stopwords_bangla.xlsx',index_col=False)

In [ ]:
from sklearn.model_selection import train_test_split
df_train,df_test= train_test_split(df, test_size=0.33, random_state=42)

In [ ]:
df_train.head()

In [ ]:
df_train = df_train.rename({'Data': 'text', 'Label': 'label'}, axis=1)
df_test = df_test.rename({'Data': 'text', 'Label': 'label'}, axis=1)

In [ ]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [ ]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['text'] = df_train['text'].apply(lambda x: preprocess(x))
df_test['text'] = df_test['text'].apply(lambda x:preprocess(x))

In [ ]:
import numpy as np
allcats = set(df_train['label'].dropna().tolist())
# allcats.add('Correct')
labeldict = {}
counter = 0
for i in allcats:
    labeldict[i] = counter
    counter += 1
labeldict

In [ ]:
def manage(x):
    if x in labeldict:
        return labeldict[x]
   
df_train['label'] = df_train['label'].apply(lambda x:manage(x))
df_test['label'] = df_test['label'].apply(lambda x:manage(x))

In [ ]:
data_no = 5

# Prepare the training data
train_texts = df_train['text'].tolist()
train_labels = df_train['label'].tolist()

test_texts = df_test['text'].tolist()
test_labels = df_test['label'].tolist()

In [ ]:
# Tokenize and encode the training texts
train_encodings = tokenizer(train_texts, truncation=True, max_length=128,
        padding='max_length', return_tensors = 'pt')

# Convert the labels to tensors
train_labels = torch.tensor(train_labels)

# Create a PyTorch dataset
train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'],
                                               train_encodings['attention_mask'],
                                               train_labels)

# Create a data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

model = model.to(device)

In [ ]:
from tqdm.notebook import tqdm
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Set the model to training mode
model.train()

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

losses = []
accuracies = []  # To store accuracy per epoch
num_epochs = 10
# Training loop
for epoch in tqdm(range(num_epochs)):  # Number of training epochs
    running_loss = 0.0
    predicted_labels = []  # To store predicted labels for accuracy calculation
    true_labels = []  # To store true labels for accuracy calculation

    for batch in tqdm(train_loader):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Convert logits to predicted labels
        _, predicted = torch.max(logits, dim=1)
        predicted_labels.extend(predicted.cpu().tolist())
        true_labels.extend(labels.cpu().tolist())

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

    # Calculate and store accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    accuracies.append(accuracy)

    print(f'Epoch {epoch + 1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {accuracy:.4f}')

# Save the model
torch.save(model.state_dict(), 'model.pth')


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def predict_labels(text):
    train_encodings = tokenizer(text, truncation=True, max_length=128,
        padding='max_length', return_tensors = 'pt')
    input_ids = train_encodings['input_ids'].to(device)
    attention_mask = train_encodings['attention_mask'].to(device)

    # Set the model to evaluation mode
    model.eval()

    # Disable gradient calculation
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)
    

    return predicted_class.item(), probabilities[:,1].item()

In [ ]:
predicted_labels = []
predicted_probs = []
for text in tqdm(test_texts):
    predicted_label, prob = predict_labels(text)
    predicted_labels.append(predicted_label)
    predicted_probs.append(prob)

# Calculate accuracy and F1 score
accuracy = accuracy_score(test_labels, predicted_labels)
# f1 = f1_score(test_labels, predicted_labels)
# roc_auc = roc_auc_score(test_labels, predicted_probs)

print('Accuracy:', accuracy)
# print('F1 Score:', f1)
# print('ROC-AUC:', roc_auc)

In [ ]:
print('Accuracy:', accuracy)

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report

print('\nThe Classification Report is as follows\n')
print(classification_report(test_labels, predicted_labels, digits = 4))